## Import libraries

In [4]:
import pdfplumber
import pandas as pd
import re

## Define the regular expressions


In [12]:
pdf_path = '/Users/elhamali/Documents/Data Projects/clf-climate-label-study/sample-3-recipe-cards.pdf'

## Open the PDF with pdfplumber

In [8]:
# Initialize the dataframe to store data
data = []

In [14]:
with pdfplumber.open(pdf_path) as pdf:
    # Loop through each page in the PDF
    for page in pdf.pages:
        # Extract text from the current page
        text = page.extract_text()
        lines = text.split('\n')
        
        # Extract specific variables based on known positions
        date = lines[2].split()[1] if len(lines) > 2 else 'Unknown Date'  
        meal_type_index = next((i for i, line in enumerate(lines) if 'Meal Type' in line), None)

        # Extract variable based on unknown position
        if meal_type_index is not None:
            meal_type = lines[meal_type_index].split()[-1]
            # Assuming the recipe note is the line immediately after meal_type
            recipe_note = lines[meal_type_index + 1] if meal_type_index + 1 < len(lines) else 'No Note'
        else:
            meal_type = 'Unknown Meal Type'
            recipe_note = 'No Note'
        
        # Use regular expressions to match and parse the text
        recipe_name_match = re.search(recipe_name_re, text)
        yield_match = re.search(yield_re, text)
        portions_match = re.search(portions_re, text)
        site = 'JHU Nolans on 33rd'
        
        # Only proceed if all parts were found
        if recipe_name_match and yield_match and portions_match:
            recipe_name = recipe_name_match.group(1)
            yield_amount = yield_match.group(1)
            portions, serving_portion_size = portions_match.groups()
            
            # Find all ingredients
            ingredients_matches = re.findall(ingredients_re, text)
            
            # Process each ingredient match
            for ingredient_match in ingredients_matches:
                ingredient, portion_used, unit_and_portion_size = ingredient_match
                portion_size = " ".join(unit_and_portion_size.split()[:-1])
                unit = unit_and_portion_size.split()[-1]
                # Add the details to the data list
                data.append({
                    'Ingredient': ingredient,
                    'Portion Used': portion_used,
                    'Portion Size': portion_size,
                    'Unit': unit,
                    'Recipe Name': recipe_name,
                    'Yield': yield_amount,
                    'Portions': portions,
                    'Serving Portion Size': serving_portion_size,
                    'Site': 'JHU Nolans on 33rd',
                    'Date': date,
                    'Recipe Note': recipe_note,
                    'Meal Type': meal_type
                })

## Convert the data into a pandas dataframe

In [15]:
df = pd.DataFrame(data)

## Save the dataframe to an Excel file

In [17]:
output_path = '/Users/elhamali/Documents/Data Projects/clf-climate-label-study/recipe-extraction-sheet.xlsx'
df.to_excel(output_path, index=False)